In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def read_data(data_path_file, result_file_path):
    route_regex = "Route [0-9]+"
    # cost_regex = "Total cost :"
    dataframe = pd.read_csv(data_path_file)
    data = dataframe.to_numpy()
    distance_data = np.append(data.copy(), np.zeros((data.shape[0], 2)), axis = 1) # [x, y, demand, centroid_index, distance_to_centroid]
    result_file = open(result_file_path, "r")
    lines = result_file.readlines()
    cluster_centroids = []  ## [x, y, current demand]
    for route_line_index in range(len(lines) - 2):
        route = lines[route_line_index]
        route_matchObj = re.match(route_regex, route, re.M|re.I)
        # print(route[len(route_matchObj.group()) + 2:].strip().split())
        city_list = list(map(int, route[len(route_matchObj.group()) + 2:].strip().split("Violation")[0].strip().split()))
        distance_data[city_list, 3] = route_line_index
        # total_demand = np.sum(data[city_list][:,2])
        cluster_coord = list(np.mean(data[city_list, :2], axis=0))
        # cluster_coord.append(total_demand)
        cluster_centroids.append(cluster_coord)
    return np.array(cluster_centroids), distance_data

In [3]:
### CALCULATE DISTANCE MATRIX ###
def calculate_distance_matrix(centroids, points):
    """
        centroids: shape(k,2)
        points: shape(m, 2)

        output : shape (m, k)
    """
    centroids_ext = np.expand_dims(centroids, axis=0)
    points_ext = np.expand_dims(points, axis=1)
    return np.sum((centroids_ext - points_ext) ** 2, axis=2) ** 0.5

In [4]:
data_path_file = "clustering\\fuzzy_c_mean\\data\\X-n106-k14.csv"
result_file_path = "clustering\\fuzzy_c_mean\\data\\X-n106-k14_output.txt"

In [5]:
cluster_centroids, distance_data = read_data(data_path_file, result_file_path)

In [6]:
capacity = 35
vehicle_num = 8

In [7]:
def remove_violate_element(sort_centroids, distance_data, capacity):
    for sort_centroid in sort_centroids:
        if sort_centroid[3] > capacity: ## If cluster is violated
            centroid_index = sort_centroid[0]
            ## Get index of element that belong to cluster
            element_index = np.where(distance_data[:, 3] == centroid_index)
            ## Get element that belong to the cluster
            element = distance_data[distance_data[:, 3] == centroid_index]
            # print(element)
            ## Sort element by their demand (ascending)
            sort_element_indices = np.argsort(element[:,2])
            ## Loop for remove all violated element
            for sort_element_index in sort_element_indices:
                if sort_centroid[3] > capacity:
                    sort_centroid[3] = sort_centroid[3] - element[sort_element_index][2]
                    element[sort_element_index][3] = -1
                else:
                    break
            ## Update element data to distance data
            distance_data[element_index] = element
        else:
            break
    return sort_centroids, distance_data

In [8]:
def post_process(cluster_centroids : np.ndarray,
                distance_data : np.ndarray, 
                capacity : int):
    
    ### Calculate total demand in each clusters ###
    centroids = None  ## CLuster index, x, y, total demand
    frame = pd.DataFrame(distance_data)
    demands = frame.groupby(3).sum()[2].to_numpy()
    centroids = np.append(
                cluster_centroids.copy(), 
                np.array([demands]).T,
                axis=1)
    centroids = np.append(
                np.expand_dims(np.arange(cluster_centroids.shape[0]), axis=0).T,
                centroids,
                axis=1
                )
    ### REMOVE POINT THAT VIOLATE OUT OF CLUSTER ###
    sort_centroids = centroids[centroids[:,3].argsort()[::-1]]
    ## element data with -1 is not clustered 
    udpated_centroids, updated_distance_data = remove_violate_element(sort_centroids, distance_data.copy(), capacity)
    udpated_centroids = udpated_centroids[udpated_centroids[:,0].argsort()]
    ### ASSIGN UN_CLUSTERED ELEMENT TO NEW CLUSTER ###
    unclustered_data_indices= np.argwhere(updated_distance_data[:, 3] == -1)
    unclustered_distance_matrix = calculate_distance_matrix(udpated_centroids[:,1:3], updated_distance_data[:, :2])
    for unclustered_data_index in np.squeeze(unclustered_data_indices.T):
        ## Check adding customer demand exceeds vehicle capacity 
        unclustered_element = updated_distance_data[unclustered_data_index]
        centroid_distance = unclustered_distance_matrix[unclustered_data_index]
        nearest_centroid_order = np.argsort(centroid_distance)
        is_violated = True
        for nearest_centroid_idx in nearest_centroid_order:
            ## Check adding customer demand exceeds vehicle capacity 
            if udpated_centroids[nearest_centroid_idx, 3] + unclustered_element[2] <= capacity:
                udpated_centroids[nearest_centroid_idx, 3] = udpated_centroids[nearest_centroid_idx, 3] + unclustered_element[2]
                is_violated = False
                updated_distance_data[unclustered_data_index, 3] = nearest_centroid_idx
                break
        if is_violated:# If all capacity is violate, assign to cluster have minimum violation (minimum current capacity)
            min_cap_centroid_idx = np.argmin(udpated_centroids[:, 3], axis = 0)
            udpated_centroids[min_cap_centroid_idx, 3] = udpated_centroids[min_cap_centroid_idx, 3] + unclustered_element[2]
            updated_distance_data[unclustered_data_index, 3] = min_cap_centroid_idx

    return centroids, udpated_centroids, updated_distance_data

In [9]:
old_centroids, udpated_centroids, updated_distance_data = post_process(cluster_centroids, distance_data, capacity)

In [10]:
old_centroids

array([[0.00000000e+00, 6.15272727e+02, 1.32000000e+02, 8.35000000e+02],
       [1.00000000e+00, 9.32500000e+02, 4.71250000e+02, 3.33000000e+02],
       [2.00000000e+00, 8.60625000e+02, 6.51500000e+02, 5.93000000e+02],
       [3.00000000e+00, 6.00272727e+02, 2.27909091e+02, 8.67000000e+02],
       [4.00000000e+00, 6.75111111e+02, 1.92666667e+02, 6.81000000e+02],
       [5.00000000e+00, 7.04666667e+02, 3.40166667e+02, 4.67000000e+02],
       [6.00000000e+00, 8.48125000e+02, 1.90250000e+02, 1.13100000e+03],
       [7.00000000e+00, 9.66000000e+02, 6.96400000e+02, 6.78000000e+02],
       [8.00000000e+00, 9.32600000e+02, 2.97600000e+02, 3.86000000e+02],
       [9.00000000e+00, 9.68200000e+02, 1.53800000e+02, 2.90000000e+02],
       [1.00000000e+01, 9.60500000e+02, 8.41000000e+02, 3.61000000e+02],
       [1.10000000e+01, 4.62000000e+02, 2.49166667e+02, 4.95000000e+02],
       [1.20000000e+01, 7.55166667e+02, 1.00500000e+02, 4.87000000e+02],
       [1.30000000e+01, 8.87000000e+02, 7.86000000e

In [11]:
udpated_centroids

array([[  0.        , 615.27272727, 132.        , 583.        ],
       [  1.        , 932.5       , 471.25      , 591.        ],
       [  2.        , 860.625     , 651.5       , 593.        ],
       [  3.        , 600.27272727, 227.90909091, 594.        ],
       [  4.        , 675.11111111, 192.66666667, 575.        ],
       [  5.        , 704.66666667, 340.16666667, 588.        ],
       [  6.        , 848.125     , 190.25      , 593.        ],
       [  7.        , 966.        , 696.4       , 575.        ],
       [  8.        , 932.6       , 297.6       , 600.        ],
       [  9.        , 968.2       , 153.8       , 594.        ],
       [ 10.        , 960.5       , 841.        , 361.        ],
       [ 11.        , 462.        , 249.16666667, 576.        ],
       [ 12.        , 755.16666667, 100.5       , 557.        ],
       [ 13.        , 887.        , 786.        , 484.        ]])

In [12]:
distance_data

array([[ 615.,  190.,   81.,    3.,    0.],
       [ 890.,  200.,   64.,    6.,    0.],
       [ 983.,  684.,   51.,    7.,    0.],
       [ 959.,  743.,   61.,    7.,    0.],
       [ 569.,  146.,   71.,    0.,    0.],
       [ 969.,  841.,   83.,   10.,    0.],
       [ 607.,  256.,   92.,    3.,    0.],
       [ 936.,  274.,   66.,    8.,    0.],
       [ 650.,  221.,   92.,    4.,    0.],
       [ 943.,  547.,   78.,    1.,    0.],
       [ 789.,  172.,   98.,    6.,    0.],
       [ 632.,  209.,   88.,    3.,    0.],
       [ 849.,  163.,   70.,    6.,    0.],
       [ 829.,  283.,   72.,    6.,    0.],
       [ 618.,  147.,   92.,    0.,    0.],
       [ 925.,  627.,   84.,    2.,    0.],
       [ 925.,  192.,   51.,    9.,    0.],
       [ 903.,  460.,   81.,    1.,    0.],
       [ 666.,  368.,   71.,    5.,    0.],
       [ 906.,  622.,   98.,    2.,    0.],
       [ 833.,  185.,   70.,    6.,    0.],
       [ 411.,  136.,   53.,   11.,    0.],
       [ 989.,  691.,   76.,    

In [13]:
updated_distance_data

array([[ 615.,  190.,   81.,    0.,    0.],
       [ 890.,  200.,   64.,    1.,    0.],
       [ 983.,  684.,   51.,    2.,    0.],
       [ 959.,  743.,   61.,    3.,    0.],
       [ 569.,  146.,   71.,    4.,    0.],
       [ 969.,  841.,   83.,    5.,    0.],
       [ 607.,  256.,   92.,    6.,    0.],
       [ 936.,  274.,   66.,    7.,    0.],
       [ 650.,  221.,   92.,    8.,    0.],
       [ 943.,  547.,   78.,    9.,    0.],
       [ 789.,  172.,   98.,   10.,    0.],
       [ 632.,  209.,   88.,   11.,    0.],
       [ 849.,  163.,   70.,   12.,    0.],
       [ 829.,  283.,   72.,   13.,    0.],
       [ 618.,  147.,   92.,    2.,    0.],
       [ 925.,  627.,   84.,    3.,    0.],
       [ 925.,  192.,   51.,    1.,    0.],
       [ 903.,  460.,   81.,    7.,    0.],
       [ 666.,  368.,   71.,   12.,    0.],
       [ 906.,  622.,   98.,    4.,    0.],
       [ 833.,  185.,   70.,   13.,    0.],
       [ 411.,  136.,   53.,    9.,    0.],
       [ 989.,  691.,   76.,    